In [ ]:
import pandas as pd
import tensorflow as tf 

train_data = pd.read_csv('/code/mnist/train.csv')
test_data = pd.read_csv('/code/mnist/test.csv')

X = train_data.drop('label', axis=1)
Y = train_data['label']

In [ ]:
X = X.astype('float32') / 255
test_data = test_data.astype('float32') / 255

Y = tf.keras.utils.to_categorical(Y, 10)

In [ ]:
train_x = X[:round(X.shape[0] * 0.8)]
check_x = X[round(X.shape[0] * 0.8):]

train_y = Y[:round(Y.shape[0] * 0.8)]
check_y = Y[round(Y.shape[0] * 0.8):]

In [ ]:
# from tensorflow.keras.layers import Dense, Dropout

# inputs = tf.keras.Input(shape=(784,))
# x_1 = Dense(256, activation='relu')(inputs)
# x_2 = Dropout(0.2)(x_1)
# outputs = Dense(10, activation='softmax')(x_2)

# model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [ ]:
train_x = train_x.values.reshape(-1, 28, 28, 1)
check_x = check_x.values.reshape(-1, 28, 28, 1)

test_data = test_data.values.reshape(-1, 28, 28, 1)

In [ ]:
train_x.shape

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten


inputs = tf.keras.Input(shape=(28, 28, 1))
x_1 = Conv2D(32, kernel_size=(5, 5), strides=1, activation='relu', padding='same')(inputs)
x_2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x_1)
x_3 = Flatten()(x_2)
x_4 = Dropout(0.4)(x_3)
outputs = Dense(10, activation='softmax')(x_4)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [ ]:
LEARNING_RATE = 0.001
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=[tf.keras.metrics.categorical_accuracy],
)

In [ ]:
train_y.shape

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=2, 
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/code/mnist/data/temp/mnist_sequential_{epoch:03d}_{val_loss:.4f}.h5',
        save_best_only=True
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir='/code/mnist/logs',
        histogram_freq=1
    ),
]

BATCH_SIZE = 128
EPOCHS = 20
VALIDATION_SPLIT = 0.2

history = model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, callbacks=callbacks)
model.save('mnist_model.h5')

In [ ]:
model.summary()

In [ ]:
model.evaluate(check_x, check_y)

- 0.9811905026435852
- 0.98416668176651 maxpooling
- 0.9783333539962769 karnel(5,5) => (3,3) & padding 'same'
- 0.9825000166893005 karnel => (3,3)
- 0.9840475916862488 drop 0.2 => 0.4

In [ ]:
submit = pd.DataFrame()
submit['ImageId'] = [i for i in range (1,test_data.shape[0]+1)]
submit['Label'] = model.predict(test_data).argmax(axis=1)

submit.to_csv('/code/mnist/submit.csv',index=False)